In [29]:
import pandas as pd
import numpy as np
from scipy.stats import zscore

# Set pandas display options for better console output
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)

print("Jupyter Notebook: Data Cleaning and Merging\n")

Jupyter Notebook: Data Cleaning and Merging



In [30]:
# load the data
tsla_df = pd.read_csv('TSLA.csv')
spy_df = pd.read_csv('SPY.csv')
vix_df = pd.read_csv('vix_daily.csv')



In [31]:
# quick check for the data
print("\nTSLA Head:")
print(tsla_df.head())
print("\nTSLA Info:")
tsla_df.info()

print("\nSPY Head:")
print(spy_df.head())
print("\nSPY Info:")
spy_df.info()

print("\nVIX Head:")
print(vix_df.head())
print("\nVIX Info:")
vix_df.info()



TSLA Head:
         Date   Open   High    Low  Close  Adj Close    Volume
0  2010-06-29  3.800  5.000  3.508  4.778      4.778  93831500
1  2010-06-30  5.158  6.084  4.660  4.766      4.766  85935500
2  2010-07-01  5.000  5.184  4.054  4.392      4.392  41094000
3  2010-07-02  4.600  4.620  3.742  3.840      3.840  25699000
4  2010-07-06  4.000  4.000  3.166  3.222      3.222  34334500

TSLA Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2956 entries, 0 to 2955
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Date       2956 non-null   object 
 1   Open       2956 non-null   float64
 2   High       2956 non-null   float64
 3   Low        2956 non-null   float64
 4   Close      2956 non-null   float64
 5   Adj Close  2956 non-null   float64
 6   Volume     2956 non-null   int64  
dtypes: float64(5), int64(1), object(1)
memory usage: 161.8+ KB

SPY Head:
                        Date        Open        High     

In [32]:
print("\n--- [3. Standardizing and Merging Data] ---")

#convert tesla to the right format (YYYY-MM-DD format)
tsla_df['Date'] = pd.to_datetime(tsla_df['Date']).dt.normalize()
tsla_df = tsla_df.set_index('Date')
# Select only the columns we need
tsla_clean = tsla_df[['Adj Close', 'Volume']].copy()
tsla_clean.columns = ['TSLA_Adj_Close', 'TSLA_Volume']

# 2. Process SPY
spy_df['Date'] = pd.to_datetime(spy_df['Date'], utc=True).dt.tz_localize(None).dt.normalize()
spy_df = spy_df.set_index('Date')
# Select only Close column
spy_clean = spy_df[['Close']].copy()
spy_clean.columns = ['SPY_Close']

# 3. Process VIX (note: VIX uses lowercase 'date' and 'close')
# Convert to datetime and normalize to date only
vix_df['Date'] = pd.to_datetime(vix_df['date']).dt.normalize()
vix_df = vix_df.set_index('Date')
# Select only close column
vix_clean = vix_df[['close']].copy()
vix_clean.columns = ['VIX_Close']

# merge data
# # 'inner' join keeps ONLY the dates where all 3 datasets have data
df = pd.concat([tsla_clean, spy_clean, vix_clean], axis=1, join='inner')

# dropping rows with NaN
nan_count = df.isnull().sum().sum()
if nan_count > 0:
    print(f"Found {nan_count} unexpected NaNs. Dropping rows.")
    df.dropna(inplace=True)

print("\n--- Merged Data Head ---")
print(df.head())
print(f"\nTotal aligned trading days: {len(df)}")
print(f"Date range: {df.index.min().date()} to {df.index.max().date()}")
print(f"\nColumns: {df.columns.tolist()}")



--- [3. Standardizing and Merging Data] ---

--- Merged Data Head ---
            TSLA_Adj_Close  TSLA_Volume  SPY_Close  VIX_Close
Date                                                         
2010-06-29           4.778     93831500  79.807274  34.130001
2010-06-30           4.766     85935500  79.049141  34.540001
2010-07-01           4.392     41094000  78.696846  32.860001
2010-07-02           3.840     25699000  78.267967  30.120001
2010-07-06           3.222     34334500  78.781113  29.650000

Total aligned trading days: 2956
Date range: 2010-06-29 to 2022-03-24

Columns: ['TSLA_Adj_Close', 'TSLA_Volume', 'SPY_Close', 'VIX_Close']


In [33]:
# --- [Cell 4: Feature Engineering (Standardized Units)] ---
print("\n--- [4. Feature Engineering (Creating Returns & Changes)] ---")

# Use log returns for price series (using Adj Close for accurate returns)
df['TSLA_Return'] = np.log(df['TSLA_Adj_Close'] / df['TSLA_Adj_Close'].shift(1))
df['SPY_Return'] = np.log(df['SPY_Close'] / df['SPY_Close'].shift(1))

# For VIX, we use the simple daily change in the close value
df['VIX_Change'] = df['VIX_Close'].diff()

# Log-transform Volume (Check Skewness first)
volume_skew = df['TSLA_Volume'].skew()
print(f"TSLA Volume Skewness: {volume_skew:.4f}")

if volume_skew > 1.0:
    print("Volume is highly skewed. Applying log transform.")
    df['TSLA_Volume_Log'] = np.log(df['TSLA_Volume'])
    df['TSLA_Volume_Change'] = df['TSLA_Volume_Log'].diff()
else:
    print("Volume is not highly skewed. Using simple percent change.")
    df['TSLA_Volume_Change'] = df['TSLA_Volume'].pct_change()

# Drop the first row, which is now NaN due to .shift() and .diff()
df = df.iloc[1:]

print("\nCreated TSLA_Return, SPY_Return, VIX_Change, and TSLA_Volume_Change.")
print("\n--- Data with new features ---")
print(df[['TSLA_Adj_Close', 'TSLA_Return', 'SPY_Close', 'SPY_Return', 'VIX_Close', 'VIX_Change', 'TSLA_Volume', 'TSLA_Volume_Change']].head())



--- [4. Feature Engineering (Creating Returns & Changes)] ---
TSLA Volume Skewness: 2.3217
Volume is highly skewed. Applying log transform.

Created TSLA_Return, SPY_Return, VIX_Change, and TSLA_Volume_Change.

--- Data with new features ---
            TSLA_Adj_Close  TSLA_Return  SPY_Close  SPY_Return  VIX_Close  VIX_Change  TSLA_Volume  TSLA_Volume_Change
Date                                                                                                                  
2010-06-30           4.766    -0.002515  79.049141   -0.009545  34.540001    0.410000     85935500           -0.087904
2010-07-01           4.392    -0.081723  78.696846   -0.004467  32.860001   -1.680000     41094000           -0.737735
2010-07-02           3.840    -0.134312  78.267967   -0.005465  30.120001   -2.740000     25699000           -0.469410
2010-07-06           3.222    -0.175470  78.781113    0.006535  29.650000   -0.470001     34334500            0.289699
2010-07-07           3.160    -0.019430  81

In [34]:
# --- [Cell 5: Drop any remaining NaN values] ---
print("\n--- [5. Cleaning up NaN values] ---")

# Drop the NaN created by the transformations
rows_before = len(df)
df.dropna(inplace=True)
rows_after = len(df)

print(f"Rows before: {rows_before}")
print(f"Rows after dropping NaN: {rows_after}")
print(f"Rows removed: {rows_before - rows_after}")

print("\n--- Clean data preview ---")
print(df.head())



--- [5. Cleaning up NaN values] ---
Rows before: 2955
Rows after dropping NaN: 2955
Rows removed: 0

--- Clean data preview ---
            TSLA_Adj_Close  TSLA_Volume  SPY_Close  VIX_Close  TSLA_Return  SPY_Return  VIX_Change  TSLA_Volume_Log  TSLA_Volume_Change
Date                                                                                                                                   
2010-06-30           4.766     85935500  79.049141  34.540001    -0.002515   -0.009545    0.410000        18.269108           -0.087904
2010-07-01           4.392     41094000  78.696846  32.860001    -0.081723   -0.004467   -1.680000        17.531373           -0.737735
2010-07-02           3.840     25699000  78.267967  30.120001    -0.134312   -0.005465   -2.740000        17.061963           -0.469410
2010-07-06           3.222     34334500  78.781113  29.650000    -0.175470    0.006535   -0.470001        17.351661            0.289699
2010-07-07           3.160     34608500  81.262375  26.

In [35]:
# --- [Cell 6: Remove Outliers (> 3 SD)] ---
print("\n--- [6. Removing Outliers] ---")

# Define the columns we will use in our regression
cols_to_clean = ['TSLA_Return', 'SPY_Return', 'VIX_Change', 'TSLA_Volume_Change']

# Calculate Z-scores
z_scores = df[cols_to_clean].apply(zscore)
abs_z_scores = np.abs(z_scores)

# Create a filter for rows where all z-scores are < 3
filtered_entries = (abs_z_scores < 3).all(axis=1)

original_rows = len(df)
df_cleaned = df[filtered_entries]
new_rows = len(df_cleaned)

print(f"Original rows: {original_rows}")
print(f"Rows removed as outliers: {original_rows - new_rows}")
print(f"Rows remaining: {new_rows}")



--- [6. Removing Outliers] ---
Original rows: 2955
Rows removed as outliers: 134
Rows remaining: 2821


In [36]:
# --- [Cell 7: Final Inspection and Save] ---
print("\n--- [7. Final Merged Dataset] ---")

# Use the cleaned dataframe from Cell 6
print("\nFinal Data Shape:")
print(f"Rows: {len(df_cleaned)}, Columns: {len(df_cleaned.columns)}")

print("\nFinal Data Head:")
print(df_cleaned.head())

print("\nFinal Data Info:")
df_cleaned.info()

print("\nDescriptive Statistics for Key Variables:")
key_vars = ['TSLA_Return', 'SPY_Return', 'VIX_Change', 'TSLA_Volume_Change']
print(df_cleaned[key_vars].describe())

# Save the complete merged dataset to CSV
df_cleaned.to_csv('cleaned_merged_data.csv')
print("\n--- SUCCESS ---")
print("Successfully saved complete merged data to 'cleaned_merged_data.csv'")
print(f"\nThe dataset includes:")
print(f"  - All original columns from TSLA, SPY, and VIX (prefixed)")
print(f"  - Calculated features: TSLA_Return, SPY_Return, VIX_Change, TSLA_Volume_Change")
print(f"  - {len(df_cleaned)} trading days from {df_cleaned.index.min().date()} to {df_cleaned.index.max().date()}")



--- [7. Final Merged Dataset] ---

Final Data Shape:
Rows: 2821, Columns: 9

Final Data Head:
            TSLA_Adj_Close  TSLA_Volume  SPY_Close  VIX_Close  TSLA_Return  SPY_Return  VIX_Change  TSLA_Volume_Log  TSLA_Volume_Change
Date                                                                                                                                   
2010-06-30           4.766     85935500  79.049141  34.540001    -0.002515   -0.009545    0.410000        18.269108           -0.087904
2010-07-01           4.392     41094000  78.696846  32.860001    -0.081723   -0.004467   -1.680000        17.531373           -0.737735
2010-07-07           3.160     34608500  81.262375  26.840000    -0.019430    0.031010   -2.809999        17.359610            0.007949
2010-07-08           3.492     38557000  82.066528  25.709999     0.099903    0.009847   -1.130001        17.467648            0.108038
2010-07-09           3.480     20253000  82.679161  24.980000    -0.003442    0.007437   